PubMed Journal Article Fetcher for Google Colab
This notebook fetches articles from specified journals for a given month using PubMed API

In [2]:
# Install required packages

!pip install biopython pandas requests

import os, pandas as pd, requests, time, warnings, json
from Bio import Entrez
from datetime import datetime, timedelta
import xml.etree.ElementTree as ET
from typing import List, Dict, Optional
import calendar
# warnings.filterwarnings('ignore')


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 29.4 MB/s eta 0:00:00


# Instructions for use

📋 INSTRUCTIONS:

1. Update the EMAIL variable with your email address (required by NCBI)
2. Adjust JOURNALS, ENT_KEYWORDS, NEGATIVE_TERMS, and LOOKBACK_DAYS to shape the search
3. Run the cells to execute main() and harvest only new PMIDs within the rolling window or the requested month

⚠️  IMPORTANT NOTES:

- Use your real email address - it’s required by NCBI’s usage policy
- Journal names should match PubMed’s format exactly
- Large queries may take several minutes to complete
- Be respectful of API rate limits
- Previously harvested PMIDs are stored in data/ent_search/seen_pmids.json to avoid duplicates
- When running via GitHub Actions, you can supply TARGET_MONTH (YYYY-MM) or YEAR + MONTH inputs to rerun a specific calendar month instead of the rolling LOOKBACK_DAYS window.

🚀 To start, run: main()


In [ ]:
# Parameters
EMAIL = os.getenv("UNPAYWALL_EMAIL", "")

JOURNALS = [
    "International Forum of Allergy & Rhinology",
    "Rhinology",
    "JAMA Otolaryngology–Head & Neck Surgery",
    "Otolaryngology–Head and Neck Surgery",
    "European Annals of Otorhinolaryngology–Head and Neck Diseases",
    "Journal of Voice",
    "American Journal of Rhinology & Allergy",
    "JARO – Journal of the Association for Research in Otolaryngology",
    "Journal of Otolaryngology–Head & Neck Surgery",
    "Laryngoscope",
    "Auris Nasus Larynx",
    "new england journal of medicine",
    "JAMA"
]

ENT_KEYWORDS = [
    "otolaryngology",
    "ENT",
    "sinus",
    "nasal",
    "larynx",
    "otology",
    "rhinology",
    "head and neck surgery"
]

NEGATIVE_TERMS = [
    "veterinary",
    "rodent",
    "mouse",
    "rat",
    "bovine",
    "porcine",
    "canine",
    "feline"
]

LOOKBACK_DAYS = int(os.getenv("LOOKBACK_DAYS", "30"))
TARGET_MONTH = os.getenv("TARGET_MONTH", "").strip()
TARGET_YEAR = os.getenv("TARGET_YEAR", "").strip()
TARGET_MONTH_NUMBER = os.getenv("TARGET_MONTH_NUMBER", "").strip()
OUTPUT_DIR = os.path.join("data", "ent_search")

ALLOW_EMPTY_HARVEST = os.getenv("ALLOW_EMPTY_HARVEST", "false").lower() == "true"


In [5]:
from datetime import datetime

RUN_STARTED_AT = datetime.now()


In [ ]:
#!/usr/bin/env python3
def load_seen_pmids(path: str) -> set:
    """Load a set of previously seen PMIDs from disk."""
    if os.path.exists(path):
        try:
            with open(path) as f:
                data = json.load(f)
                if isinstance(data, list):
                    return set(map(str, data))
        except Exception as exc:
            print(f"⚠️  Could not load seen PMIDs: {exc}")
    return set()

def save_seen_pmids(path: str, pmids: set) -> None:
    """Persist a set of PMIDs to disk."""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, "w") as f:
        json.dump(sorted(pmids), f, indent=2)

def compute_requested_window(target_month: str, target_year: str, target_month_number: str):
    """Return a (start_date, end_date) tuple for a requested calendar month."""
    target_month = (target_month or "").strip()
    target_year = (target_year or "").strip()
    target_month_number = (target_month_number or "").strip()

    if target_month:
        try:
            start_date = datetime.strptime(target_month, "%Y-%m")
        except ValueError as exc:
            raise ValueError(f"TARGET_MONTH must be YYYY-MM; received '{target_month}'") from exc
        end_day = calendar.monthrange(start_date.year, start_date.month)[1]
        end_date = start_date.replace(day=end_day, hour=23, minute=59, second=59)
        return start_date, end_date

    if target_year and target_month_number:
        try:
            year_int = int(target_year)
            month_int = int(target_month_number)
            start_date = datetime(year_int, month_int, 1)
        except ValueError as exc:
            raise ValueError(
                f"TARGET_YEAR and TARGET_MONTH_NUMBER must form a valid date; received '{target_year}-{target_month_number}'"
            ) from exc
        end_day = calendar.monthrange(year_int, month_int)[1]
        end_date = start_date.replace(day=end_day, hour=23, minute=59, second=59)
        return start_date, end_date

    return None

class PubMedFetcher:

    def __init__(self, email: str):
        """Initialize with email for API requests"""
        self.email = email
        Entrez.email = email
        # Be respectful to NCBI servers
        self.request_delay = 0.34  # ~3 requests per second max

    def search_articles(
        self,
        journals: List[str],
        keywords: List[str],
        negative_terms: List[str],
        lookback_days: int,
        start_date_override: Optional[datetime] = None,
        end_date_override: Optional[datetime] = None,
    ):
        """
        Search for articles in specified journals with keyword filters within a rolling or fixed window.

        Args:
            journals: List of journal names
            keywords: Positive keyword filters
            negative_terms: Terms to exclude from results
            lookback_days: Number of days to include in the rolling window
            start_date_override: Explicit start date to use instead of the rolling window
            end_date_override: Explicit end date to use instead of the rolling window

        Returns:
            Tuple of (List of PubMed IDs, start_date_str, end_date_str)
        """
        end_date = end_date_override or datetime.now()
        start_date = start_date_override or end_date - timedelta(days=lookback_days)

        start_date_str = start_date.strftime("%Y/%m/%d")
        end_date_str = end_date.strftime("%Y/%m/%d")

        journal_query = " OR ".join([f'"{journal}"[Journal]' for journal in journals]) if journals else ""
        keyword_query = " OR ".join([f'"{kw}"[Title/Abstract]' for kw in keywords]) if keywords else ""
        negative_query = " AND ".join([f'NOT "{term}"[Title/Abstract]' for term in negative_terms]) if negative_terms else ""

        query_parts = []
        if journal_query:
            query_parts.append(f"({journal_query})")
        if keyword_query:
            query_parts.append(f"({keyword_query})")
        if negative_query:
            query_parts.append(negative_query)
        query_parts.append(f"({start_date_str}[PDAT] : {end_date_str}[PDAT])")

        final_query = " AND ".join(query_parts)

        try:
            print(f"Querying PubMed with: {final_query}")
            handle = Entrez.esearch(db="pubmed", term=final_query, datetype="pdat", retmax=100000)
            record = Entrez.read(handle)
            handle.close()
            time.sleep(self.request_delay)

            search_results = record
            id_list = search_results["IdList"]
            print(f"Found {len(id_list)} articles")
            return id_list, start_date_str, end_date_str

        except Exception as e:
            print(f"Error searching PubMed: {e}")
            return [], start_date_str, end_date_str

    def fetch_article_details(self, pmid_list: List[str]) -> List[Dict]:
        """
        Fetch detailed information for articles

        Args:
            pmid_list: List of PubMed IDs

        Returns:
            List of article dictionaries
        """
        articles = []
        batch_size = 200  # Process in batches to avoid overwhelming API

        for i in range(0, len(pmid_list), batch_size):
            batch = pmid_list[i:i + batch_size]
            print(f"Fetching batch {i//batch_size + 1} with {len(batch)} articles")
            try:
                handle = Entrez.efetch(db="pubmed", id=batch, rettype="xml", retmode="text")
                records = Entrez.read(handle)
                handle.close()
                time.sleep(self.request_delay)

                for article in records.get("PubmedArticle", []):
                    parsed_article = self.parse_article(article)
                    if parsed_article:
                        articles.append(parsed_article)

            except Exception as e:
                print(f"Error fetching details for batch {i//batch_size + 1}: {e}")
                continue

        return articles

    def parse_article(self, article) -> Optional[Dict]:
        """
        Parse article details from PubMed XML
        """
        try:
            journal = article.find(".//Journal/Title").text or "N/A"
            title_elem = article.find(".//ArticleTitle")
            title = title_elem.text if title_elem is not None else "N/A"

            pmid_elem = article.find(".//PMID")
            pmid = pmid_elem.text if pmid_elem is not None else ""

            authors_list = article.findall(".//Author")
            authors = []
            for author in authors_list:
                last = author.findtext("LastName", default="")
                fore = author.findtext("ForeName", default="")
                full_name = " ".join([fore, last]).strip()
                if full_name:
                    authors.append(full_name)
            authors_str = ", ".join(authors)

            pub_date_elem = article.find(".//PubDate")
            year = pub_date_elem.findtext("Year", default="N/A") if pub_date_elem is not None else "N/A"
            month = pub_date_elem.findtext("Month", default="N/A") if pub_date_elem is not None else "N/A"
            day = pub_date_elem.findtext("Day", default="") if pub_date_elem is not None else ""
            pub_date_str = f"{year}/{month}/{day}" if day else f"{year}/{month}"

            doi_elem = article.find(".//ArticleId[@IdType='doi']")
            doi = doi_elem.text if doi_elem is not None else "N/A"

            volume = article.findtext(".//JournalIssue/Volume", default="") or "N/A"
            issue = article.findtext(".//JournalIssue/Issue", default="") or "N/A"
            pages = article.findtext(".//Pagination/MedlinePgn", default="") or "N/A"

            abstract_elem = article.find(".//Abstract/AbstractText")
            if abstract_elem is not None:
                if list(article.findall(".//Abstract/AbstractText")):
                    abstract_parts = []
                    for abs_part in article.findall(".//Abstract/AbstractText"):
                        label = abs_part.get("Label", "")
                        text = abs_part.text or ""
                        if label:
                            abstract_parts.append(f"{label}: {text}")
                        else:
                            abstract_parts.append(text)
                    abstract = " ".join(abstract_parts)
                else:
                    abstract = abstract_elem.text or ""
            else:
                abstract = ""

            return {
                "PMID": pmid,
                "Title": title,
                "Authors": authors_str,
                "Journal": journal,
                "Publication_Date": pub_date_str,
                "Volume": volume,
                "Issue": issue,
                "Pages": pages,
                "DOI": doi,
                "Abstract": abstract[:500] + "..." if len(abstract) > 500 else abstract  # Truncate long abstracts
            }

        except Exception as e:
            print(f"Error parsing article: {e}")
            return None

def main():

    print("=== PubMed Journal Article Fetcher ===")

    print(f"Email: {EMAIL}")
    print(f"Journals: {', '.join(JOURNALS)}")
    print(f"Keyword filters: {', '.join(ENT_KEYWORDS)}")
    print(f"Excluding terms: {', '.join(NEGATIVE_TERMS)}")

    requested_window = compute_requested_window(TARGET_MONTH, TARGET_YEAR, TARGET_MONTH_NUMBER)
    if requested_window:
        start_dt, end_dt = requested_window
        print(
            f"Requested month window: {start_dt.strftime('%Y-%m-%d')} to {end_dt.strftime('%Y-%m-%d')} (from TARGET_MONTH/TARGET_YEAR+TARGET_MONTH_NUMBER)"
        )
    else:
        end_dt = datetime.now()
        start_dt = end_dt - timedelta(days=LOOKBACK_DAYS)
        print(f"Rolling window: last {LOOKBACK_DAYS} days ({start_dt.strftime('%Y-%m-%d')} to {end_dt.strftime('%Y-%m-%d')})")
    print("-" * 50)

    # Validate email
    if EMAIL == "your.email@example.com":
        print("⚠️  Please update the EMAIL variable with your actual email address!")
        print("This is required by NCBI's API usage policy.")
        return

    # Initialize fetcher
    fetcher = PubMedFetcher(EMAIL)

    # Search for articles within the requested window
    print("🔍 Searching for articles...")
    pmid_list, start_date, end_date = fetcher.search_articles(
        JOURNALS,
        ENT_KEYWORDS,
        NEGATIVE_TERMS,
        LOOKBACK_DAYS,
        start_dt,
        end_dt,
    )

    if not pmid_list:
        print("❌ No articles found matching the criteria.")
        print(f"Summary: discovered {len(pmid_list)} PMIDs between {start_date} and {end_date}.")
        if ALLOW_EMPTY_HARVEST:
            print("⚠️  Empty harvest allowed via ALLOW_EMPTY_HARVEST flag; exiting without failure.")
            return
        raise RuntimeError("No articles found for the configured search window.")

    seen_pmids_path = os.path.join(OUTPUT_DIR, "seen_pmids.json")
    seen_pmids = load_seen_pmids(seen_pmids_path)
    if seen_pmids:
        print(f"Loaded {len(seen_pmids)} previously harvested PMIDs.")

    new_pmids = [pmid for pmid in pmid_list if pmid not in seen_pmids]
    print(f"PMIDs to fetch after filtering seen set: {len(new_pmids)}")

    if not new_pmids:
        print("⚠️  No new PMIDs to process within this window.")
        print(f"Summary: discovered {len(pmid_list)} PMIDs, seen {len(seen_pmids)}, new {len(new_pmids)} between {start_date} and {end_date}.")
        if ALLOW_EMPTY_HARVEST:
            print("⚠️  Empty harvest allowed via ALLOW_EMPTY_HARVEST flag; exiting without failure.")
            return
        raise RuntimeError("No new PMIDs to process after filtering seen set.")

    # Fetch article details
    print(f"
📖 Fetching details for {len(new_pmids)} new articles...")
    articles = fetcher.fetch_article_details(new_pmids)

    if not articles:
        print("❌ Failed to fetch article details.")
        return

    # Create DataFrame
    df = pd.DataFrame(articles)

    # Display results
    print(f"
✅ Successfully retrieved {len(articles)} articles!")
    print(f"
Columns: {', '.join(df.columns.tolist())}")

    # Show first few rows
    print(f"
First 5 articles:")
    print(df.head().to_string(max_colwidth=50))

    # Save outputs
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    window_label = f"{start_date.replace('/', '-')}_to_{end_date.replace('/', '-')}"
    csv_path = os.path.join(OUTPUT_DIR, f"ent_raw_results_{window_label}.csv")
    df.to_csv(csv_path, index=False)

    updated_seen_pmids = seen_pmids | set(new_pmids)
    save_seen_pmids(seen_pmids_path, updated_seen_pmids)
    print(f"Updated seen PMIDs saved to: {seen_pmids_path}")

    json_path = os.path.join(OUTPUT_DIR, "ent_all_results.json")
    df.to_json(json_path, orient="records", force_ascii=False, indent=2)

    # Display summary statistics
    print(f"
📊 Summary:")
    print(f"Total new articles: {len(articles)}")
    print(f"Unique journals: {df['Journal'].nunique()}")
    print(f"Articles per journal:")
    journal_counts = df['Journal'].value_counts()
    for journal, count in journal_counts.head(10).items():
        print(f"  • {journal}: {count}")

    print(f"
💾 Raw CSV saved to: {csv_path}")
    print(f"💾 ENT results JSON saved to: {json_path}")

    return df


In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
df = main()
if df is not None:
    print(f"\nSaved results to {OUTPUT_DIR}")
